In [311]:
import numpy as np
from ase.io import read, write
from ase.neighborlist import NeighborList, NewPrimitiveNeighborList
import sys
import json

In [270]:
# Initialize Julia
import julia
julia.install()

In [271]:
# Create a Julia instance
jl = julia.Julia(compiled_modules=False)

In [285]:
import julia.Main as Main
Main.include("rings.jl")
Main.include("IOPiper.jl")

<PyCall.jlwrap Main.IOPiper>

In [291]:
def make_receiver(io):
            def receiver(s):
                io.write(s)
                io.flush()
            return receiver

Main.IOPiper.pipe_std_outputs(
    make_receiver(sys.stdout),
    make_receiver(sys.stderr))

Main.IOPiper.io_test()

Testing redirect


# ASE way of generating neighbourlist

In [319]:
ats = read('structures/aSi_100k_relaxed_amorphous.xyz', '-1')

In [324]:
ats = read('structures/varMQ_11Ks_4096_500K_min.xyz', '-1')

In [274]:
ats = read('aSi_500atom_11Ks_annealed.xyz', '-1')

In [325]:
nl = NeighborList(cutoffs=2.85, self_interaction=False, bothways=True,
                  primitive=NewPrimitiveNeighborList)
nl.update(ats)

True

In [326]:
neighs = [nl.get_neighbors(i)[0] for i in range(len(ats))]

In [322]:
import csv

# Write the data to a file
with open('nodlnkd_100k.txt', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    for row in neighs:
        csvwriter.writerow(row+1)

In [277]:
len(ats)

512

In [278]:
ats.cell[0,0]

21.848854069638442

In [327]:
def halton_sequence(dim, num_points, ats):
    vol = ats.get_volume()
    def halton_base_seq(base):
        seq = []
        for i in range(num_points):
            x = (20**3 / vol)**(1/3) * 0.2
            f = 1.0 / base
            while i > 0:
                x += f * (i % base)
                i //= base
                f /= base
            seq.append(x)
        return seq
    
    sequence = []
    for d in range(dim):
        base = 2 + d  # Use different prime bases for each dimension
        sequence.append(halton_base_seq(base))
    
    return np.transpose(sequence)

def place_points_in_cube(num_points, ats):
    dim = 3  # Number of dimensions
    points = halton_sequence(dim, num_points, ats)
    return points

# Define the number of points you want to place
vol = ats.get_volume()
num_points = int(np.ceil((vol/(20**3)))) + 2
print('Num refnodes requested: ', num_points)

# Place points maximally separated in the periodic unit cube
points = place_points_in_cube(num_points, ats)

spos = ats.get_scaled_positions()
refnodes = []

for v in points:
    diff = spos - v
    ref = np.argmin(np.linalg.norm(diff, axis=1))
    refnodes.append(ref+1) # Julia indexing
    
refnodes.append(1)
print('Refnodes chosen: ', refnodes)


Num refnodes requested:  13
Refnodes chosen:  [206, 826, 3304, 1809, 2405, 2866, 1346, 3802, 2542, 919, 1549, 4034, 585, 1]


In [1]:
(144.36 - 50 - 50) / 7

6.337142857142859

In [302]:
Main.include("rings.jl")

<PyCall.jlwrap Main.rings>

In [305]:
Main.rings.test_func()

Testing


In [307]:
rs = Main.rings.py_ring_statistics(len(ats), neighs, refnodes)

RuntimeError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: MethodError: no method matching py_ring_statistics(::Int64, ::Vector{Vector{Int64}}, ::Vector{Any})

Closest candidates are:
  py_ring_statistics(::Int64, !Matched::PyArray, !Matched::PyArray)
   @ Main.rings ~/data/julia_rings/rings.jl:392
  py_ring_statistics(::Int64, !Matched::PyArray, !Matched::PyArray, !Matched::Int64)
   @ Main.rings ~/data/julia_rings/rings.jl:392
  py_ring_statistics(::Int64, !Matched::PyArray, !Matched::PyArray, !Matched::Int64, !Matched::Int64)
   @ Main.rings ~/data/julia_rings/rings.jl:392
  ...

Stacktrace:
 [1] invokelatest(::Any, ::Any, ::Vararg{Any}; kwargs::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
   @ Base ./essentials.jl:816
 [2] invokelatest(::Any, ::Any, ::Vararg{Any})
   @ Base ./essentials.jl:813
 [3] _pyjlwrap_call(f::Function, args_::Ptr{PyCall.PyObject_struct}, kw_::Ptr{PyCall.PyObject_struct})
   @ PyCall ~/.julia/packages/PyCall/ilqDX/src/callb

In [33]:
nodlnkd

array([[  1,   8],
       [  1,   1],
       [  1,   3],
       ...,
       [512, 236],
       [512, 244],
       [512, 505]])

In [22]:
result = jl.eval("rings.ring_statistics()")

JuliaError: Exception 'MethodError: no method matching ring_statistics()

Closest candidates are:
  ring_statistics(!Matched::Any, !Matched::Any)
   @ Main.rings ~/data/julia_rings/rings.jl:205
  ring_statistics(!Matched::Any, !Matched::Any, !Matched::Any)
   @ Main.rings ~/data/julia_rings/rings.jl:205
  ring_statistics(!Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any)
   @ Main.rings ~/data/julia_rings/rings.jl:205
  ...
' occurred while calling julia code:
rings.ring_statistics()

In [264]:
with open('rings_out.json', 'r') as f:
    rings_out = json.load(f)

In [265]:
rings_out[5][0]

[1, 3, 27, 37, 1352, 12]

In [266]:
r_set = [i for i in rings_out[9]]
srcs = [i[0] for i in rings_out[9]]

In [267]:
r_set[0]


[55, 1416, 1417, 56, 12569, 12566, 84, 89, 79, 80]

In [268]:
from collections import defaultdict

# Initialize dictionaries to store duplicates and non-duplicates
duplicate_sets = defaultdict(list)
non_duplicate_sets = defaultdict(list)
uns_dup_sets = defaultdict(list)
dup_cts = defaultdict(int)
set_srcs = defaultdict(set)

# Iterate through the array of sets
for index, su in enumerate(r_set):
    s = frozenset(su)
    if s in duplicate_sets:
        duplicate_sets[s].append(index)
        set_srcs[s].add(srcs[index])
        dup_cts[s] += 1
    elif s in non_duplicate_sets:
        duplicate_sets[s] = non_duplicate_sets[s]
        dup_cts[s] = 2
        duplicate_sets[s].append(index)
        set_srcs[s].add(srcs[index])
        del non_duplicate_sets[s]
    else:
        non_duplicate_sets[s] = [index]
        set_srcs[s] = {srcs[index]}
        uns_dup_sets[s] = su

# Extract sets with duplicates and non-duplicates
sets_with_duplicates = [key for key in duplicate_sets]
sets_without_duplicates = [key for key in non_duplicate_sets]

# Print results
print("Sets with duplicates: ", len(sets_with_duplicates))
for s in sets_with_duplicates:
    sorted_s = sorted(s)
    unsorted_s = s
    print(sorted_s, dup_cts[s], " Srcs \n", sorted(set_srcs[s]), " ", unsorted_s, "\n")

print("\nSets without duplicates:", len(sets_without_duplicates))
for s in sets_without_duplicates:
    sorted_s = sorted(s)
    print(sorted_s, dup_cts[s], " Srcs \n", sorted(set_srcs[s]), " ", uns_dup_sets[s], "\n")


Sets with duplicates:  89
[55, 56, 79, 80, 84, 89, 1416, 1417, 12566, 12569] 10  Srcs 
 [55, 56, 79, 80, 84, 89, 1416, 1417, 12566, 12569]   frozenset({89, 1416, 1417, 79, 80, 84, 12566, 55, 56, 12569}) 

[593, 594, 630, 631, 660, 661, 13068, 13073, 13111, 13136] 10  Srcs 
 [593, 594, 630, 631, 660, 661, 13068, 13073, 13111, 13136]   frozenset({631, 13068, 13136, 13073, 594, 593, 660, 661, 630, 13111}) 

[1452, 1453, 1484, 1485, 1495, 1516, 1525, 1535, 1547, 1550] 10  Srcs 
 [1452, 1453, 1484, 1485, 1495, 1516, 1525, 1535, 1547, 1550]   frozenset({1547, 1484, 1485, 1452, 1453, 1516, 1550, 1525, 1495, 1535}) 

[5988, 5989, 6022, 93490, 93491, 93521, 93543, 93550, 93551, 93552] 10  Srcs 
 [5988, 5989, 6022, 93490, 93491, 93521, 93543, 93550, 93551, 93552]   frozenset({5988, 5989, 6022, 93543, 93550, 93551, 93552, 93521, 93490, 93491}) 

[6167, 6196, 6197, 6198, 6199, 6213, 6222, 6232, 6233, 6234] 10  Srcs 
 [6167, 6196, 6197, 6198, 6199, 6213, 6222, 6232, 6233, 6234]   frozenset({6213, 6